# Web Scraping for Indeed.com & Predicting Salaries

**Problem Statement:**

Many different variables can influence the salary of a job in data science. One main variable is the city in which the job is located. Websites such as Indeed.com provide a wealth of information on jobs, including salary and location. This code uses web scraping and logistic regression to predict whether a salary will be high or low depending on the location of the job. 

In [539]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [542]:
#Functions to parse html on Indeed.com 
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''
        

def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

def get_location_from_result(result):
    return  extract_text(result.find('span', {'class' : 'location'}))

def get_summary_from_result(result):
    return  extract_text(result.find('span', {'class' : 'summary'}))

def get_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()
        
    return None

In [544]:
#Function that substitutes city names and iterates through results to gain enough salary information to analyze
#Code has been commented out so that it does not run unintentionally


#url = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
#rows = []

#for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle']):
 #   for start in range(10, 8000, 10):
  #      r = requests.get(url.format(city, start))
   #     soup = BeautifulSoup(r.content)
    #    results = soup.findAll('div', { "class" : "result" })
     #   for result in results:
      #      if result:
       #         row = {}
        #        row['title'] = get_title_from_result(result)
         #       row['company'] = get_company_from_result(result)
          #      row['summary'] = get_summary_from_result(result)
           #     row['salary'] = get_salary_from_result(result)
            #    row['city'] = city
             #   rows.append(row)


data = pd.DataFrame.from_records(rows)

salary_data = data[data.salary.notnull()]
salary_data = salary_data[~(salary_data.salary.astype('str').str.contains('hour'))]

In [138]:
salary_data.head()

NameError: name 'salary_data' is not defined

In [ ]:
#commented out because running line again would cause csv to be empty unless I rerun Beautifulsoup...as I found out
#salary_data.to_csv('../assets/Project4/IndeedSalaryData.csv', encoding='utf-8',index=False)

In [3]:
sdf = pd.read_csv('../Assets/Project4/IndeedSalaryData.csv')

In [4]:

sdf.head(100)

,city,company,salary,summary,title
0,San+Francisco,Workbridge Associates,"$120,000 - $165,000 a year","Familiarity with data visualization, web analy...",Senior Level Data Scientist
1,San+Francisco,Workbridge Associates,"$125,000 - $135,000 a year","Experience with big data stack technologies, s...",Mid-Level Data Scientist
2,San+Francisco,MarkMonitor,"$180,000 a year","Data skills (SQL, Hive, Pig). Applying machine...",Data Scientist
3,San+Francisco,Mines.io,"$80,000 - $120,000 a year",We are looking for a data scientist/developer ...,Full-Stack Data Scientist
4,San+Francisco,Selby Jennings,"$160,000 a year",Integrate data and control APIs utilzing 3rd p...,Business Intelligence/Data Scientist at VC Bay...
5,San+Francisco,Workbridge Associates,"$130,000 - $175,000 a year",3+ years as a Data Scientist required. FinTech...,Senior Data Scientist (FinTech)
6,San+Francisco,Workbridge Associates,"$120,000 - $150,000 a year",There is a small team of Data Engineers and Da...,Mid-Level Data Engineer
7,San+Francisco,Workbridge Associates,"$150,000 - $180,000 a year",A well-known San Francisco gaming company is l...,Senior Data Scientist
8,San+Francisco,California State University,"$50,400 - $60,000 a year",Compile and analyze data for written reports a...,"Administrative Analyst/Specialist, Exempt I - ..."
9,San+Francisco,University of California Berkeley,"$3,811 - $3,889 a month","Must be able to work in a greenhouse, manage d...","Staff Research Associate, PMB"


**Data Dictionary**

|city|City where job is located|
|company|Company that listed the job|
|salary|Salary for the listed job|
|summary|Job description|
|title|Job title|

In [5]:
#Splitting salary on '-' so that we no longer have a range of salaries
sdf2 = sdf['salary'].apply(lambda x: pd.Series(x.split('-'))) 
sdf2.columns = ['MinSal','MaxSal']
print sdf2.dtypes
sdf2.head()


MinSal    object
MaxSal    object
dtype: object


,MinSal,MaxSal
0,"$120,000","$165,000 a year"
1,"$125,000","$135,000 a year"
2,"$180,000 a year",NaN
3,"$80,000","$120,000 a year"
4,"$160,000 a year",NaN


In [6]:
#Filling MaxSal column with value from MinSal if there is a null
sdf2['MaxSal'].fillna(sdf2['MinSal'], inplace=True)
sdf2.head()



,MinSal,MaxSal
0,"$120,000","$165,000 a year"
1,"$125,000","$135,000 a year"
2,"$180,000 a year","$180,000 a year"
3,"$80,000","$120,000 a year"
4,"$160,000 a year","$160,000 a year"


In [7]:
#Cleaning extra string from salary columns in dataframe
sdf2['MinSal'] = sdf2['MinSal'].astype(str)
sdf2['MaxSal'] = sdf2['MaxSal'].astype(str)


sdf2['MinSal'] = sdf2['MinSal'].str.replace(",","")    
sdf2['MinSal'] = sdf2['MinSal'].str.replace("$","")
sdf2['MinSal'] = sdf2['MinSal'].str.replace("a year","")
sdf2['MinSal'] = sdf2['MinSal'].str.replace("a month","")
sdf2['MinSal'] = sdf2['MinSal'].str.replace("a day","")
sdf2['MinSal'] = sdf2['MinSal'].str.replace("a week","")


sdf2['MaxSal'] = sdf2['MaxSal'].str.replace(",","")
sdf2['MaxSal'] = sdf2['MaxSal'].str.replace("$","")
sdf2['MaxSal'] = sdf2['MaxSal'].str.replace("a year","")
sdf2['MaxSal'] = sdf2['MaxSal'].str.replace("a month","")
sdf2['MaxSal'] = sdf2['MaxSal'].str.replace("a day","")
sdf2['MaxSal'] = sdf2['MaxSal'].str.replace("a week","")


print sdf2.shape

sdf2.head()

(1729, 2)


,MinSal,MaxSal
0,120000,165000
1,125000,135000
2,180000,180000
3,80000,120000
4,160000,160000


In [8]:
sdf3 = sdf.join(sdf2)
sdf3.shape

(1729, 7)

In [9]:
sdf3.head()

,city,company,salary,summary,title,MinSal,MaxSal
0,San+Francisco,Workbridge Associates,"$120,000 - $165,000 a year","Familiarity with data visualization, web analy...",Senior Level Data Scientist,120000,165000
1,San+Francisco,Workbridge Associates,"$125,000 - $135,000 a year","Experience with big data stack technologies, s...",Mid-Level Data Scientist,125000,135000
2,San+Francisco,MarkMonitor,"$180,000 a year","Data skills (SQL, Hive, Pig). Applying machine...",Data Scientist,180000,180000
3,San+Francisco,Mines.io,"$80,000 - $120,000 a year",We are looking for a data scientist/developer ...,Full-Stack Data Scientist,80000,120000
4,San+Francisco,Selby Jennings,"$160,000 a year",Integrate data and control APIs utilzing 3rd p...,Business Intelligence/Data Scientist at VC Bay...,160000,160000


In [10]:
#Converting to int so that we can average columns and no longer have a range
sdf3['MinSal'] = sdf3['MinSal'].astype(int)
sdf3['MaxSal'] = sdf3['MaxSal'].astype(int)



sdf3["AvgSal"] = sdf3[["MinSal", "MaxSal"]].mean(axis=1)

sdf3 = sdf3[sdf3['AvgSal'] >= 30000] #getting rid of data where salary was in terms of weeks, days, months, or 
                                        #we don't care about cause it is too low.

sdf3.drop(['salary','MinSal','MaxSal'], axis=1 ,inplace=True)

sdf3['city'] = sdf3['city'].str.replace("+", " ")


print sdf3.shape
sdf3.head()


(884, 5)


,city,company,summary,title,AvgSal
0,San Francisco,Workbridge Associates,"Familiarity with data visualization, web analy...",Senior Level Data Scientist,142500.0
1,San Francisco,Workbridge Associates,"Experience with big data stack technologies, s...",Mid-Level Data Scientist,130000.0
2,San Francisco,MarkMonitor,"Data skills (SQL, Hive, Pig). Applying machine...",Data Scientist,180000.0
3,San Francisco,Mines.io,We are looking for a data scientist/developer ...,Full-Stack Data Scientist,100000.0
4,San Francisco,Selby Jennings,Integrate data and control APIs utilzing 3rd p...,Business Intelligence/Data Scientist at VC Bay...,160000.0


In [49]:
sdf3.drop_duplicates(keep='first', inplace=True)
sdf3.shape

(185, 7)

In [50]:
#Separating salaries into bins, then labeling according to the bin the salary falls in 
bins = [0.0, 70000.0, 500000.0]
groupNames = ['Low','High']
category = pd.cut(sdf3['AvgSal'], bins, labels=groupNames)


In [51]:
sdf3['salCategory'] = category
sdf3.to_csv('../Assets/Project4/TableauData.csv')

In [52]:
#Making dummy variables for salary from High/Low bins
sdf3['Salary'] = sdf3['salCategory'].map({'Low': 0, 'High':1})
sdf3.head()

,city,company,summary,title,AvgSal,salCategory,Salary
0,San Francisco,Workbridge Associates,"Familiarity with data visualization, web analy...",Senior Level Data Scientist,142500.0,High,1
1,San Francisco,Workbridge Associates,"Experience with big data stack technologies, s...",Mid-Level Data Scientist,130000.0,High,1
2,San Francisco,MarkMonitor,"Data skills (SQL, Hive, Pig). Applying machine...",Data Scientist,180000.0,High,1
3,San Francisco,Mines.io,We are looking for a data scientist/developer ...,Full-Stack Data Scientist,100000.0,High,1
4,San Francisco,Selby Jennings,Integrate data and control APIs utilzing 3rd p...,Business Intelligence/Data Scientist at VC Bay...,160000.0,High,1


In [53]:
#making dummy ranks for the 5 cities examined
dummy_ranks = pd.get_dummies(sdf3['city'])
dummy_ranks.shape

(185, 5)

In [54]:

data = sdf3[['Salary']].join(dummy_ranks)


In [55]:
data.head()

,Salary,Austin,Chicago,New York,San Francisco,Seattle
0,1,0.0,0.0,0.0,1.0,0.0
1,1,0.0,0.0,0.0,1.0,0.0
2,1,0.0,0.0,0.0,1.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0
4,1,0.0,0.0,0.0,1.0,0.0


In [56]:
data.to_csv('../Assets/Project4/FinalData.csv', encoding='utf-8',index=False)

In [57]:
#reading in as new dataframe and adding a column so that we can see the intercept of the logistic regression line
Fdata = pd.read_csv('../Assets/Project4/FinalData.csv')
Fdata['Intercept'] = 1
Fdata.head()

,Salary,Austin,Chicago,New York,San Francisco,Seattle,Intercept
0,1,0.0,0.0,0.0,1.0,0.0,1
1,1,0.0,0.0,0.0,1.0,0.0,1
2,1,0.0,0.0,0.0,1.0,0.0,1
3,1,0.0,0.0,0.0,1.0,0.0,1
4,1,0.0,0.0,0.0,1.0,0.0,1


In [58]:
import statsmodels.api as sm
import pylab as pl
import numpy as np
from sklearn.cross_validation import train_test_split
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [59]:
print pd.crosstab(sdf3['Salary'], sdf3['city'], rownames=['HiLow'])

city   Austin  Chicago  New York  San Francisco  Seattle
HiLow                                                   
0           3        2        22              4        3
1           5       30        66             33       17


This crosstab of the salaries converted to either a High(1) or Low(0) values for each city. High indicates that a job has a salary of over 100,000 dollars annually, whereas Low indicates that the job pays less than 100,000 dollars annually.
We can clearly see that New York has the most overall jobs with salaries from our sample, and the most low and high paying jobs. San Francisco has the best ratio of high to low paying jobs and Austin has the fewest jobs in either category. 

In [60]:

FdataColumns = Fdata.columns

y = Fdata.Salary
X = Fdata[FdataColumns[2:]]
print X.shape
print y.shape
#Fdata.head()


(185, 5)
(185,)


In [61]:
Fdata.corr()

,Salary,Austin,Chicago,New York,San Francisco,Seattle,Intercept
Salary,1.000000,-0.104961,0.143211,-0.162841,0.097694,0.030369,NaN
Austin,-0.104961,1.000000,-0.097227,-0.202495,-0.106299,-0.074017,NaN
Chicago,0.143211,-0.097227,1.000000,-0.435597,-0.228665,-0.159222,NaN
New York,-0.162841,-0.202495,-0.435597,1.000000,-0.476240,-0.331611,NaN
San Francisco,0.097694,-0.106299,-0.228665,-0.476240,1.000000,-0.174078,NaN
Seattle,0.030369,-0.074017,-0.159222,-0.331611,-0.174078,1.000000,NaN
Intercept,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
#Using train, test, split to split the data into a test sample of 30% and a train sample of 70%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=20)

In [63]:
#logistic regression model
logit = sm.Logit(y_train, X_train)
result = logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.443619
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Salary   No. Observations:                  129
Model:                          Logit   Df Residuals:                      124
Method:                           MLE   Df Model:                            4
Date:                Fri, 01 Jul 2016   Pseudo R-squ.:                 0.05371
Time:                        07:50:57   Log-Likelihood:                -57.227
converged:                       True   LL-Null:                       -60.475
                                        LLR p-value:                    0.1650
=================================================================================
                    coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
Chicago           2.0149      1.065      1.893      0.058        -0.072     4.101
New York          0.9234      0.822      1.123      0.261        -0.688     2.535
San Francisco     1.8326      0.978      1.874      0.061        -0.084     3.750
Seattle           2.0149      1.297      1.553      0.120        -0.528     4.558
Intercept         0.2877      0.764      0.377      0.706        -1.209     1.785
=================================================================================
"""

This summary table displays data for each city that was generated by our model. The coeffiecents discussed are log odds and not probabilities or odds ratios.

**Chicago**

This table indicates that the coefficient for Chicago is 2.01. This demonstrates that there is a positive relationship between job salary in Chicago and the baseline for our model, which is Austin. This means that you are more likely to have a high paying job in Chicago, the p-value is low enough for the model to argued to be significant, however, we can see that the confidence interval contains zero, so this may not be the best variable for predicting whether a job salary is going to be high or low as it may not be much better than chance.

**New York**

The coefficient for New York is 0.92. It is positive, but small, so that means you are barely more likely to get a higher paying job in New York versus Austin. It has a high p-value, so it is not a significant indicator of salary. Additionally, the confidence interval contains zero, so it is not good predictor for salaries.

**San Francisco**

San Francisco has a positive coefficient, so you are more likely to get a high paying job in San Francisco than Austin.
The p-value is arguably significant, but the confidence interval contains zero, so it also is not the greatest predictor in this model.

**Seattle**

Seattle has a positive coefficient as well, so you are also more likely to get a high paying job there. However, the p-value is not very significant and the confidence interval contains zero, so also probably not the greatest predictor for the model. 

**Intercept**

This represents the logistic model with no predictors. It indicates that the model would predict a higher paying job in the absence of the other variables(in Austin). The p-value is very high and the confidence interval contains zero, so this variable is not much better than chance at predicting whether a job would be high or low paying.

In [101]:
print np.exp(result.params)

params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

Chicago          7.500000
New York         2.517857
San Francisco    6.250000
Seattle          7.500000
Intercept        1.333333
dtype: float64
                   2.5%      97.5%        OR
Chicago        0.930868  60.427483  7.500000
New York       0.502528  12.615436  2.517857
San Francisco  0.919021  42.504471  6.250000
Seattle        0.589772  95.375877  7.500000
Intercept      0.298416   5.957371  1.333333


Above is the odds ratio(OR) and how it is effected by small and large confidence intervals. The odds ratio tells the fold change in the relationship between the salary in a given city and the salaries in Austin. For example, in San Francisco, you would have an increase of 6.25 in high salary jobs per 1.00 increase in high salary jobs in Austin. All of the cities outside of Austin have a high odds ratio compared to Austin.

In [102]:
X_test["actualSal"] = y_test

dfTrain = X_test

print dfTrain.columns

Index([u'Chicago', u'New York', u'San Francisco', u'Seattle', u'Intercept',
       u'actualSal', u'predictedSal', u'GuessSal'],
      dtype='object')


/Users/AlexStrahan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [103]:
dfTrain['predictedSal'] = result.predict( dfTrain[ dfTrain.columns[0:5] ] )

/Users/AlexStrahan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [104]:
dfTrain.head()

,Chicago,New York,San Francisco,Seattle,Intercept,actualSal,predictedSal,GuessSal
63,0.0,0.0,0.0,1.0,1,0,0.909091,1
176,1.0,0.0,0.0,0.0,1,1,0.909091,1
46,0.0,0.0,0.0,1.0,1,1,0.909091,1
28,0.0,0.0,1.0,0.0,1,1,0.892857,1
172,1.0,0.0,0.0,0.0,1,1,0.909091,1


In [105]:
threshold = 0.8
dfTrain['GuessSal'] = [0 if x < threshold else 1 for x in dfTrain['predictedSal']]

/Users/AlexStrahan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [106]:
print pd.crosstab(dfTrain['actualSal'], dfTrain['GuessSal'], rownames=['actual'])

GuessSal   0   1
actual          
0          8   3
1         20  25


Here we have a made a confusion matrix of job salaries as predicted from our logit model. Our model is actually relatively good at predicting whether a salary will be high, but not very good at predicting whether a salary will be low. We get a large percentage of true positives to false positives, but a very low percentage of the true negatives to the false negatives. Our model is not picking out many of the true positives, even if we vary the the threshold for the prediction model. Our threshold right now is very high, but if we lower it very much it does not pick out very many, if any, of the true negatives.

In [107]:
ytrue=dfTrain['actualSal']
ypred=dfTrain['GuessSal']
precision_score(ytrue,ypred)

0.8928571428571429

This is the precision score. This score tells us the fraction of scores that are relevant by taking the true positives over the false positives and the true positives. It is a ratio of the guesses that were right over the guesses that it was possible to get right. We have a high precision score. It tells us that our model is not spitting out very many false positives compared to our predicted true positives. This is not telling us the whole story in this case.

In [99]:
recall_score(ytrue,ypred)

0.55555555555555558

This is the recall score. This score displays the true positive results over the true positives and false negatives. The recall score lets us know how many predicted positives we got right out of all of the actual positives. Our recall score is not good. We only get half of the total actual positives correct. This is basically no better than random chance.

In [100]:
accuracy_score(ytrue,ypred)

0.5892857142857143

We have a low accuracy score. The accuracy score tells the total correct predictions over the total possible correct predictions. The previous measures only look at correct positive predictions, while this takes into account all predictions. Our model produces a large amount of false negatives, which hurts both our recall and accuracy. the larg number of false negatives means that even though we have a low number of false positives, our model is only slightly better than pure chance at determining whether a salary will be high or low.

**Summary**

This model is not very good at predicting whether or not a job will be high paying in the five cities that are in the model. It is only slightly better than blind luck. Other methods of regression, such as K nearest neighbor, were not able to be used in this project due to time constraints and the many categorical variables that we had in this data. In the future, k nearest neighbor would be a good complimentary approach, but requires custom functions for categorical variables that were not able to be performed. 

One limitation of our model is that we only looked at one categorical variable. In the future the model could be strengthened by adding variables such as specific keywords in the job title or the job summary. While location is a good predictor, there was also a limited number of job listings that contained salary information, so we may also have an unintentionally biased model.

There were also a lot of duplicates in the data which lowered our predictive power as we had to get rid of them and had a much smaller sample size. This could be addressed through better web scraping code that could prevent the accumulation of duplicate data, or we could simply run the web scraper for more results to account for the duplications. Overall, this model is insufficient to reliably predict salary and needs to be optimized to be useful.

The cell below contains a link to a tableau visualization that may help better explore the data:

[Salary Data](../Assets/Project4/Salaryvisualization.twb)